# Tesla Stock Analysis

## About this Project and Me

Hello, my name is Joseph Kemper. I am a Business Management student with an emphasis in Computer Programming. My goal behind choosing this project is to demonstrate what I am capable of and to build my skills for future projects. Future projects I have planned involve a large amount of data analysis and this project will serve as a good stepping stone to my long term goals. 

Goal of Analysis: Find what if any patterns that exist in Tesla Stock performance.

# Data Preparation

In [4]:
import pandas
import matplotlib.pyplot as plot
%matplotlib inline

# Load data into DataFrame
raw_data = pandas.read_csv('RawData.csv')

# Make Copy of data to clean
cleaned_data = raw_data

# Prep list of columns to remove dollar sign from
columns_to_clean = ["Close/Last","Open","High","Low"]

# Loop over columns and remove all dollar signs and set all numbers as float
for column in columns_to_clean:
    cleaned_data[column] = cleaned_data[column].replace({'\$':''}, regex=True).astype(float)

cleaned_data["Date"] = pandas.to_datetime(cleaned_data["Date"])

# Save the cleaned data to a new CSV file
cleaned_data.to_csv("CleanedData.csv", index=False)

# Get the summary statistics
summary = cleaned_data.describe()

# Print the summary
print(summary)

                                Date   Close/Last        Volume         Open  \
count                           2516  2516.000000  2.516000e+03  2516.000000   
mean   2018-11-01 12:19:27.567567360    90.346463  1.141232e+08    90.378079   
min              2013-11-01 00:00:00     8.033300  1.065415e+07     7.958700   
25%              2016-05-03 18:00:00    15.488975  6.623726e+07    15.518825   
50%              2018-10-30 12:00:00    21.321650  9.307683e+07    21.264700   
75%              2021-05-03 06:00:00   190.947450  1.340551e+08   191.469950   
max              2023-10-31 00:00:00   409.970000  9.140809e+08   411.470000   
std                              NaN   107.398548  7.681395e+07   107.498260   

              High          Low  
count  2516.000000  2516.000000  
mean     92.434231    88.214102  
min       8.181300     7.740000  
25%      15.751825    15.235850  
50%      21.644300    20.922350  
75%     196.258325   186.032500  
max     414.496600   405.666600  
std    

# Data Preparation

In [6]:
prepped_data = cleaned_data
# Create column containing the day of the week
prepped_data["DayOfWeek"] = prepped_data["Date"].dt.day_name()
# Create column containing the name of the month
prepped_data["Month"] = prepped_data["Date"].dt.month_name()
# Create column containing month abbreviation
prepped_data["MonthAbbr"] = prepped_data["Month"].str.slice(0,3)
# Create column containing the week of the year
prepped_data ["WeekNumber"] = prepped_data["Date"].dt.isocalendar().week
# Create column containing the calendar quarter
prepped_data ["Quarter"] = prepped_data["Date"].dt.quarter
# Create column containing day of the month
prepped_data ["DayOfMonth"] = prepped_data["Date"].dt.day

prepped_data.to_csv("PreppedData.csv", index=False)

## Initial Data Visualization